In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\CHCFPP01\Guest\MRajabi\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import scipy
from matplotlib.gridspec import GridSpec
from mailmerge import MailMerge
import shutil
import openpyxl
from scipy.optimize import curve_fit
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-12-19 11:20:55


In [2]:
Project_Dir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment'
SourceDir = Project_Dir + '\\IRIS_Original'
OutputDir = Project_Dir + '\\IRIS_Cleaned'
GDB = Project_Dir + '\\CrashData_Cleaned\\IL_SafetyDB.gdb'
Int_GDB = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment\Intersection_Original\Int_Original.gdb'
Int_FC = Int_GDB + '\\Intersection2014_361999_CleanedUpFall2018'
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-12-19 11:20:55


In [3]:
print(strftime("%Y-%m-%d %H:%M:%S"))
Int_DF = hsmpy3.common.FCtoDF(Int_FC,readGeometry=True)
print(Int_DF.shape)
Int_DF = Int_DF[~Int_DF.PeerGroupText_CH2M_Updated_PGs_TCTs.isin(['State - Rural, Not an Intersection'])]
Int_DF.TRAF_CONT = Int_DF.TRAF_CONT.astype(str)
Int_DF.TRAF_CONT.loc[Int_DF.TRAF_CONT==' '] = '9'
Int_DF.TRAF_CONT = Int_DF.TRAF_CONT.fillna('9')
Int_DF.LEG_COUNT = Int_DF.LEG_COUNT.astype(int,errors='ignore')
Int_DF['X'] = [s.firstPoint.X for s in Int_DF.Shape]
Int_DF['Y'] = [s.firstPoint.Y for s in Int_DF.Shape]
Int_DF.index = pd.MultiIndex.from_tuples([(i,m) for i,m in zip(Int_DF.INV,Int_DF.STA)],names=['RID','MP'])
Int_DF.sort_index()
Int_DF = Int_DF[['LEG_COUNT','TRAF_CONT','MajorRoadN', 'MinorRoadN', 'PeerGroupText_CH2M_Updated_PGs_TCTs','X','Y']]
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-11-26 15:32:41
(361999, 28)


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


2018-11-26 15:43:52


In [4]:
Int_DF.PeerGroupText_CH2M_Updated_PGs_TCTs.value_counts().sort_index()

Local - Rural, All-Way Stop Control         1546
Local - Rural, Minor Leg Stop Control      31017
Local - Rural, Signalized Intersection       347
Local - Rural, Undetermined                67702
Local - Urban, All-Way Stop Control         6924
Local - Urban, Minor Leg Stop Control      62938
Local - Urban, Signalized Intersection      8740
Local - Urban, Undetermined               136553
State - Rural, All-Way Stop Control          210
State - Rural, Minor Leg Stop Control      17781
State - Rural, Signalized Intersection       108
State - Rural, Undetermined                   56
State - Urban, All-Way Stop Control          522
State - Urban, Minor Leg Stop Control      21099
State - Urban, Signalized Intersection      6256
State - Urban, Undetermined                  199
Name: PeerGroupText_CH2M_Updated_PGs_TCTs, dtype: int64

In [5]:
Int_DF.TRAF_CONT.value_counts().sort_index()

0     22692
1     26416
2      7572
3      1339
4      1544
5      1863
6      1653
7      1228
8     10167
9    179047
A    102479
B       142
N      5569
Y       287
Name: TRAF_CONT, dtype: int64

In [6]:
Int_DF.LEG_COUNT.value_counts().sort_index()

3    245853
4    115766
5       303
6        74
8         2
Name: LEG_COUNT, dtype: int64

In [7]:
print(strftime("%Y-%m-%d %H:%M:%S"))
def TCT_Conv(t1,pg):
    tct = 6 #Unknown
    if pg.find('Minor Leg Stop Control')>0:
        tct = 1 #minor stop
    elif pg.find('All-Way Stop Control')>0:
        tct = 2 #all stop
    elif pg.find('Signalized Intersection')>0:
        tct = 3 #signal
    else:
        if t1 in ['1','3','A','B']:
            tct = 1
        elif t1 in ['2','4']:
            tct = 2
        elif t1 in ['5','6','7','8']:
            tct = 3
        elif t1 in ['Y']:
            tct = 4
        elif t1 in ['0']:
            tct = 5
        elif t1=='N':
            tct = None
    inv_st = None #unknown
    if tct==1 and t1 in ['1','3']:
        inv_st = 1 #Inv stops
    if tct==1 and t1 in ['A','B']:
        inv_st = 0 #X stops
    has_flash = None
    if tct in [1,2] and t1 in ['3','B','4']:
        has_flash = 1 #has flashing
    if tct in [1,2] and t1 in ['1','A','2']:
        has_flash = 0 #has flashing
    mp = None
    if tct in [3] and t1 in ['7','8']:
        mp = 1 #has flashing
    if tct in [3] and t1 in ['5','6']:
        mp = 0 #has flashing
    ac = None
    if tct in [3] and t1 in ['6','8']:
        ac = 1 #has flashing
    if tct in [3] and t1 in ['5','7']:
        ac = 0 #has flashing
    return([tct,inv_st,has_flash,mp,ac])
Int_DF['TCON_TYP']     = [TCT_Conv(t,p)[0] for t,p in zip(Int_DF.TRAF_CONT,Int_DF.PeerGroupText_CH2M_Updated_PGs_TCTs)]
Int_DF['INV_RT_STOPS'] = [TCT_Conv(t,p)[1] for t,p in zip(Int_DF.TRAF_CONT,Int_DF.PeerGroupText_CH2M_Updated_PGs_TCTs)]
Int_DF['HAS_FLASH']    = [TCT_Conv(t,p)[2] for t,p in zip(Int_DF.TRAF_CONT,Int_DF.PeerGroupText_CH2M_Updated_PGs_TCTs)]
Int_DF['MULTI_PHASE']  = [TCT_Conv(t,p)[3] for t,p in zip(Int_DF.TRAF_CONT,Int_DF.PeerGroupText_CH2M_Updated_PGs_TCTs)]
Int_DF['ACTUATED']     = [TCT_Conv(t,p)[4] for t,p in zip(Int_DF.TRAF_CONT,Int_DF.PeerGroupText_CH2M_Updated_PGs_TCTs)]
pd.DataFrame(Int_DF.TCON_TYP.value_counts(dropna=False).sort_index())

,TCON_TYP
1.0,132852
2.0,9202
3.0,15453
4.0,274
5.0,21851
6.0,177357
NaN,5009


In [8]:
print(strftime("%Y-%m-%d %H:%M:%S"))
def IRIS_TCT(t1):
        if t1 in ['1','3','A','B']:
            tct = 'Minor Stop (1,3,A,B)'
        elif t1 in ['2','4']:
            tct = 'All-way Stop (2,4)'
        elif t1 in ['5','6','7','8']:
            tct = 'Signal (5,6,7,8)'
        elif t1 in ['Y']:
            tct = 'Yeild (Y)'
        elif t1 in ['0']:
            tct = 'No Device (0)'
        elif t1=='N':
            tct = 'Not an Intersection (N)'
        elif t1=='9':
            tct = 'Undetermined (9)'
        return(tct)
def PG_TCT(pg):
    if pg.find('Minor Leg Stop Control')>0:
        tct = 'Minor Stop' #minor stop
    elif pg.find('All-Way Stop Control')>0:
        tct = 'All-way Stop' #all stop
    elif pg.find('Signalized Intersection')>0:
        tct = 'Signal' #signal
    else:    
        tct = 'Undetermined'
    return(tct)
Int_DF['TCT_based_on_IRIS'] = [IRIS_TCT(s) for s in Int_DF.TRAF_CONT]
Int_DF['TCT_based_on_PG'] = [PG_TCT(s) for s in Int_DF.PeerGroupText_CH2M_Updated_PGs_TCTs]
df=pd.DataFrame(Int_DF.groupby(['TCT_based_on_PG','TCT_based_on_IRIS']).size())
df = df.unstack()
df.columns = df.columns.droplevel(0)
df = df[['All-way Stop (2,4)', 'Minor Stop (1,3,A,B)', 'Signal (5,6,7,8)', 'Yeild (Y)', 'No Device (0)', 'Not an Intersection (N)', 'Undetermined (9)']]
df = df.fillna(0)
df= df.astype(int)
df

TCT_based_on_IRIS,"All-way Stop (2,4)","Minor Stop (1,3,A,B)","Signal (5,6,7,8)",Yeild (Y),No Device (0),Not an Intersection (N),Undetermined (9)
TCT_based_on_PG,,,,,,,
All-way Stop,8956,149,33,11,3,9,41
Minor Stop,127,129631,210,2,828,508,1529
Signal,33,579,14666,0,10,43,120
Undetermined,0,17,2,274,21851,5009,177357


In [12]:
print(strftime("%Y-%m-%d %H:%M:%S"))
Int_DF_Cleaned = Int_DF[['LEG_COUNT', 'TCON_TYP', 'INV_RT_STOPS', 'HAS_FLASH', 'MULTI_PHASE', 'ACTUATED','MajorRoadN', 'MinorRoadN','X','Y']]
Int_DF_Cleaned = Int_DF_Cleaned[~pd.isnull(Int_DF_Cleaned.TCON_TYP)]
for f in ['TCON_TYP','INV_RT_STOPS','HAS_FLASH','MULTI_PHASE','ACTUATED']:
    Int_DF_Cleaned.loc[:,f] = Int_DF_Cleaned[f].replace(np.NaN,None).astype(int,errors='ignore')
Int_DF_Cleaned.to_csv(Project_Dir + '\\Intersection_Cleaned\\Int_Points_Cleaned.csv')
print(Int_DF_Cleaned.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-11-26 15:46:06
(356989, 10)
2018-11-26 15:46:18


In [16]:
print(strftime("%Y-%m-%d %H:%M:%S"))
hsmpy3.il.AddDomainsToGDB(GDB,replace=False)
from hsmpy3.il import domains
from hsmpy3 import common
GDBDomains = [d.name for d in arcpy.da.ListDomains(GDB)]
Int_Points_FC = GDB + '\\Int_Points'
try:
    arcpy.Delete_management(Int_Points_FC)
except:
    pass
arcpy.CreateFeatureclass_management(out_path=GDB,out_name='Int_Points',geometry_type='POINT',has_z='ENABLED',has_m='ENABLED')
arcpy.AddField_management(Int_Points_FC,'IID','TEXT')
Int_Fields = ['LEG_COUNT', 'TCON_TYP','HAS_FLASH', 'MULTI_PHASE', 'ACTUATED','MajorRoadN', 'MinorRoadN']
for f in Int_Fields:
        if getattr(domains,f)['name'] in GDBDomains:
            arcpy.AddField_management(in_table=Int_Points_FC,
                                  field_name=getattr(domains,f)['name'],
                                  field_type = getattr(domains,f)['type'],
                                  field_alias = getattr(domains,f)['alias'],
                                  field_domain=getattr(domains,f)['name']
                                 )
        else:
            arcpy.AddField_management(in_table=Int_Points_FC,
                                  field_name=getattr(domains,f)['name'],
                                  field_type = getattr(domains,f)['type'],
                                  field_alias = getattr(domains,f)['alias']
                                 )
ic = arcpy.InsertCursor(Int_Points_FC)
for i,r in Int_DF_Cleaned.iterrows():
        row = ic.newRow()
        row.setValue('IID','{}_{:0.3f}'.format(i[0],i[1]))
        for f in Int_Fields:
            if not pd.isnull(r[f]):
                row.setValue(f,r[f])
        if not math.isnan(r.X) and not math.isnan(r.Y):
            Pt =  arcpy.PointGeometry(arcpy.Point(r.X,r.Y),common.NAD1983IL)
            row.shape = Pt
        ic.insertRow(row)
del ic
del row
print('[{}]  - Intersection Points {}'.format(strftime("%Y-%m-%d %H:%M:%S"),str(arcpy.GetCount_management(Int_Points_FC))))

2018-11-26 16:30:56
[2018-11-26 16:30:56] Add Domains
[2018-11-26 16:30:56]  - TCON_TYP
[2018-11-26 16:31:04]  - TRAF_CONT
[2018-11-26 16:31:21] Done!
[2018-11-26 16:38:25]  - Intersection Points 356989


In [3]:
for year in range(2014,2015):
    hsmpy3.il.CON_OverlayIntersections(Project_Dir,
                             HSMPY_PATH,
                             GDB + '\\Int_Points',
                             GDB + '\\Route_{}'.format(year),
                             GDB + '\\Att_{}'.format(year),
                             year)
    time.sleep(10)